# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [4]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [5]:
bigram_phrases = Phrases(sentences)

In [6]:
type(bigram_phrases.vocab)

dict

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [11]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [12]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [13]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

## Entrainement d'un premier modèle Word2Vec sur ce corpus (window = 5 ; min_count = 5))

In [126]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et 5 après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 24min 46s
Wall time: 13min 12s


### Sauver le modèle dans un fichier

In [127]:
outfile = f"../../data/tp3modèle1.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [128]:
model = Word2Vec.load("../../data/tp3modèle1.model")

### Calculer la similarité entre deux termes

In [129]:
model.wv.similarity("homme", "femme")

0.6530492

In [130]:
model.wv.similarity("bruxelles", "capitale")

0.28434867

In [131]:
model.wv.similarity("football", "sport")

0.7751446

### Chercher les mots les plus proches d'un terme donné

In [132]:
model.wv.most_similar("energie", topn=10)

[('efficacite', 0.8020390272140503),
 ('ignorance', 0.794345498085022),
 ('abnegation', 0.7877367734909058),
 ('exces', 0.7864693999290466),
 ('impuissance', 0.7857227325439453),
 ('inertie', 0.785113513469696),
 ('habilete', 0.7830312848091125),
 ('indifference', 0.7785846590995789),
 ('action', 0.7782384157180786),
 ('indulgence', 0.7774134874343872)]

In [133]:
model.wv.most_similar("capitale", topn=10)

[('principaute', 0.8992559313774109),
 ('peninsule', 0.8890219330787659),
 ('mecque', 0.8710858225822449),
 ('colonie_belge', 0.86016446352005),
 ('crete', 0.8509145379066467),
 ('mer_rouge', 0.8480837941169739),
 ('region_parisienne', 0.8446575403213501),
 ('ville', 0.8353541493415833),
 ('rhenanie', 0.8318886160850525),
 ('ruhr', 0.8301903605461121)]

In [134]:
model.wv.most_similar("colonie", topn=10)

[('guerro', 0.8754188418388367),
 ('marine_marchande', 0.8579181432723999),
 ('ruhr', 0.8459063172340393),
 ('communaute', 0.8456894159317017),
 ('guerre', 0.8295210003852844),
 ('collectivite', 0.82573002576828),
 ('demobilisation', 0.8230893611907959),
 ('nation', 0.8143597841262817),
 ('prohibition', 0.8128385543823242),
 ('rhenanie', 0.8122228384017944)]

### Faire des recherches complexes à travers l'espace vectoriel

In [135]:
print(model.wv.most_similar(positive=['sport', 'cyclisme'], negative=['football']))

[('rallye', 0.8281869888305664), ('club', 0.8214699029922485), ('ski', 0.8057788610458374), ('bridge', 0.7888575792312622), ('golf', 0.7766138911247253), ('marathon', 0.7751936316490173), ('motocyclisme', 0.7718915939331055), ('patinage', 0.7684836983680725), ('festival', 0.767212450504303), ('automobilisme', 0.7575165033340454)]


In [136]:
print(model.wv.most_similar(positive=['petrole', 'charbon'], negative=['economie']))

[('ciment', 0.7864272594451904), ('bris', 0.7456074953079224), ('dechargement', 0.7232266068458557), ('bois', 0.7160535454750061), ('rail', 0.7079124450683594), ('gravier', 0.707025408744812), ('coke', 0.7046268582344055), ('pignon', 0.7007346749305725), ('reservoir', 0.7006763219833374), ('puits', 0.7004013061523438)]


In [137]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['bruxelles']))

[('berlin', 0.8665355443954468), ('moscou', 0.863474428653717), ('rome', 0.8593438267707825), ('budapest', 0.821623682975769), ('vienne', 0.8211926221847534), ('tokio', 0.8130141496658325), ('pekin', 0.8039390444755554), ('teheran', 0.7862898111343384), ('stockholm', 0.7855962514877319), ('madrid', 0.7807698845863342)]


 # Exploration Modèle numéro 2 (window = 5 ; min_count = 25)

Entrainer, sauver et charger le modèle 2

In [104]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et 5 après le mot observé
    min_count=25, # On ignore les mots qui n'apparaissent pas au moins 25 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 23min 21s
Wall time: 14min 35s


In [105]:
outfile = f"../../data/tp3modèle2.model"
model.save(outfile)

model = Word2Vec.load("../../data/tp3modèle2.model")

Calculer la similarité entre deux termes

In [106]:
model.wv.similarity("homme", "femme")

0.5857733

In [107]:
model.wv.similarity("bruxelles", "capitale")

0.24837849

In [108]:
model.wv.similarity("football", "sport")

0.7631886

Chercher les mots les plus proches d'un terme

In [109]:
model.wv.most_similar("energie", topn=10)

[('efficacite', 0.8164990544319153),
 ('imprecision', 0.7957242131233215),
 ('habilete', 0.7945934534072876),
 ('impetuosite', 0.786982536315918),
 ('action', 0.7868802547454834),
 ('obstination', 0.7846607565879822),
 ('atome', 0.7793523669242859),
 ('agressivite', 0.778461217880249),
 ('indifference', 0.7738246917724609),
 ('avarice', 0.7710710167884827)]

In [110]:
model.wv.most_similar("capitale", topn=10)

[('principaute', 0.9143218994140625),
 ('colonie_belge', 0.8777433037757874),
 ('peninsule', 0.8536405563354492),
 ('region', 0.8454762697219849),
 ('region_parisienne', 0.8452200889587402),
 ('crete', 0.8390171527862549),
 ('ruhr', 0.8371900320053101),
 ('population', 0.8371796607971191),
 ('mer_rouge', 0.8345614075660706),
 ('ville', 0.8335004448890686)]

In [111]:
model.wv.most_similar("colonie", topn=10)

[('guerro', 0.8351351618766785),
 ('communaute', 0.8318026065826416),
 ('recherche_scientifique', 0.8177210688591003),
 ('population', 0.8173548579216003),
 ('nation', 0.8164836764335632),
 ('marine_marchande', 0.8137192726135254),
 ('ruhr', 0.8076481819152832),
 ('sarre', 0.8048380613327026),
 ('collectivite', 0.804188072681427),
 ('decentralisation', 0.802216112613678)]

Faire des recherches complexes à travers l'espace vectoriel

In [112]:
print(model.wv.most_similar(positive=['sport', 'cyclisme'], negative=['football']))

[('rallye', 0.8030558228492737), ('ski', 0.7991294860839844), ('patinage', 0.7675231099128723), ('club', 0.7660090923309326), ('festival_international', 0.7624330520629883), ('touriste', 0.7603012323379517), ('marathon', 0.7564849853515625), ('bridge', 0.75617516040802), ('lotus', 0.7492101192474365), ('motocyclisme', 0.7474668622016907)]


In [113]:
print(model.wv.most_similar(positive=['petrole', 'charbon'], negative=['economie']))

[('ciment', 0.8163164854049683), ('dechargement', 0.7656125426292419), ('vapeur', 0.7519423961639404), ('metal', 0.73738032579422), ('plancher', 0.7354918718338013), ('jambon', 0.7318801283836365), ('puits', 0.7134420871734619), ('goulot', 0.7128924131393433), ('reservoir', 0.7123075723648071), ('gaz_naturel', 0.708698570728302)]


In [114]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['bruxelles']))

[('rome', 0.8769578337669373), ('moscou', 0.8763126730918884), ('berlin', 0.8735169768333435), ('tokio', 0.8544323444366455), ('budapest', 0.8541219830513), ('vienne', 0.848977267742157), ('teheran', 0.8438239693641663), ('pekin', 0.8349310159683228), ('beyrouth', 0.8215287923812866), ('ankara', 0.8096815943717957)]


 # Exploration Modèle numéro 3 (window = 15 ; min_count = 10)

Entrainer, sauver et charger le modèle 3

In [115]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=15, # La taille du "contexte", ici 15 mots avant et 15 après le mot observé
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 10 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 26min 32s
Wall time: 13min 48s


In [116]:
outfile = f"../../data/tp3modèle3.model"
model.save(outfile)

model = Word2Vec.load("../../data/tp3modèle3.model")

Calculer la similarité entre deux termes

In [117]:
model.wv.similarity("homme", "femme")

0.60208446

In [118]:
model.wv.similarity("bruxelles", "capitale")

0.165795

In [119]:
model.wv.similarity("football", "sport")

0.77865374

Chercher les mots les plus proches d'un terme

In [120]:
model.wv.most_similar("energie", topn=10)

[('faiblesse', 0.8286552429199219),
 ('vigueur', 0.8121653199195862),
 ('inutilite', 0.7910152077674866),
 ('force', 0.7889838218688965),
 ('inertie', 0.787100613117218),
 ('apathie', 0.7767216563224792),
 ('action', 0.7766621708869934),
 ('impuissance', 0.7761543989181519),
 ('organisation_sociale', 0.77167809009552),
 ('vitalite', 0.7690632343292236)]

In [121]:
model.wv.most_similar("capitale", topn=10)

[('principaute', 0.8655822277069092),
 ('colonie_belge', 0.7873440980911255),
 ('peninsule', 0.7849032282829285),
 ('contree', 0.7683951258659363),
 ('peripherie', 0.7673091888427734),
 ('region', 0.7616746425628662),
 ('liaison_avec', 0.7574202418327332),
 ('partie_occidentale', 0.7543638348579407),
 ('banlieue', 0.7537333965301514),
 ('population', 0.7529217600822449)]

In [122]:
model.wv.most_similar("colonie", topn=10)

[('marine_marchande', 0.7861441373825073),
 ('corporation', 0.7816519737243652),
 ('transaction', 0.7761610150337219),
 ('cession', 0.7638018131256104),
 ('demobilisation', 0.7574828863143921),
 ('collectivite', 0.7505149245262146),
 ('recherche_scientifique', 0.746623694896698),
 ('caisse_autonome', 0.745442807674408),
 ('wehrmacht', 0.7453480958938599),
 ('priorite_absolue', 0.7437680959701538)]

Faire des recherches complexes à travers l'espace vectoriel

In [123]:
print(model.wv.most_similar(positive=['sport', 'cyclisme'], negative=['football']))

[('rallye', 0.808153510093689), ('club', 0.8042554259300232), ('olub', 0.7906923294067383), ('annuelle_epreuve', 0.7674909830093384), ('coupe', 0.7663635015487671), ('ciub', 0.7620835900306702), ('trophee', 0.7611589431762695), ('motocross', 0.7581794261932373), ('challenge', 0.756144106388092), ('automobilisme', 0.748543381690979)]


In [124]:
print(model.wv.most_similar(positive=['petrole', 'charbon'], negative=['economie']))

[('dechargement', 0.7855389714241028), ('chargement', 0.7737162113189697), ('wagon', 0.7684531807899475), ('naphte', 0.7533102631568909), ('ciment', 0.7505702972412109), ('puits', 0.7452909350395203), ('dragueur', 0.7449766993522644), ('mouillage', 0.7402570247650146), ('port', 0.7354591488838196), ('coke', 0.7289832830429077)]


In [125]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['bruxelles']))

[('moscou', 0.8732405304908752), ('berlin', 0.8701534867286682), ('rome', 0.8568610548973083), ('vienne', 0.8105735778808594), ('madrid', 0.80129075050354), ('aux_indes', 0.7861269116401672), ('washington', 0.7822923064231873), ('geneve', 0.7786237001419067), ('tokio', 0.7748748660087585), ('ankara', 0.774321436882019)]
